In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
html_code = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
code = BeautifulSoup(html_code)
table = code.find('table').find_all('tr')
table_cleaned = [table[i].text.strip().split('\n') for i in range(len(table))]

In [3]:
can_postal_codes = pd.DataFrame(table_cleaned[1:], columns=table_cleaned[0]) 
can_postal_codes.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
can_postal_codes = can_postal_codes[can_postal_codes.Borough != 'Not assigned']

In [5]:
can_postal_codes.set_index(['Postcode', 'Borough'],inplace=True)
can_postal_codes = can_postal_codes.groupby(level=['Postcode', 'Borough'], sort=False).agg(', '.join)
can_postal_codes.reset_index(inplace=True)
can_postal_codes.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [6]:
replace_index = can_postal_codes[can_postal_codes['Neighbourhood'] == 'Not assigned'].index
can_postal_codes['Neighbourhood'].replace('Not assigned', can_postal_codes['Borough'].loc[replace_index], inplace=True)

1. got html code of wikipedia's page
2. read the table
3. filtered 'Not assigned' boroughs
4. Replaced all 'Not assighned' neighbourhoods by borough name

In [7]:
can_postal_codes.shape

(103, 3)

In [8]:
can_postal_codes.to_csv('Postal Codes.csv', index=False)